# BOTPT Data detide DepthDifference Day

#### This method is detailed here: https://www.pmel.noaa.gov/eoi/rsn/diffs.html

In [ ]:
%reset -f

In [ ]:
# Load map of Axial caldera showing locations of BOTPT instruments (red circles)
from IPython.display import Image
Image(url = "https://www.pmel.noaa.gov/eoi/rsn/Axial-2017-OOI-caldera-ed-sm.png")

In [ ]:
#Zoomed in map of Axial caldera
from IPython.display import Image
Image(url = "https://www.pmel.noaa.gov/eoi/rsn/Axial-2017-OOI-zoom-ed-sm.png")

### API Information Setup

In [ ]:
USERNAME = 'OOIAPI-J97520T1AYPUNI'
TOKEN =  'TEMP-TOKEN-YD01XSNDIO57MP'

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import datetime
import os
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from matplotlib import pyplot
from pylab import rcParams
import pickle as pk
import gc
import requests 
import matplotlib.gridspec as gridspec
import netCDF4 as nc

#### For mad help with Pandas https://chrisalbon.com
#### For all sorts of great repos https://github.com/rabernat/research_computing.git

#### Read Downsampled BOTPT Data - I have made pickle files downsampleing the data one data point per day

In [ ]:
with open('/home/jovyan/data/botpt/bottom_pressure1day_F11.pkl', 'rb') as E:
    botpt_data = pk.load(E)
df_botpt = pd.DataFrame(botpt_data)
df_botpt['bottom_pressure'] = df_botpt['bottom_pressure'].astype(float)
df_botpt['depth']=df_botpt['bottom_pressure'].astype(float) * 0.670
#MJ03F_cal_depths = [MJ03F_pressure * 0.0670 for MJ03F_pressure in MJ03F_pressure]
#list comprehention
epoch= [i.timestamp() for i in df_botpt.index.to_pydatetime()]
df_botpt['epoch'] = epoch
df_botpt= df_botpt.sort_index()

In [ ]:
with open('/home/jovyan/data/botpt/bottom_pressure1day_E11.pkl', 'rb') as E:
    botpt_data = pk.load(E)
df_botptE = pd.DataFrame(botpt_data)
df_botptE['bottom_pressure'] = df_botptE['bottom_pressure'].astype(float)
df_botptE['depth']=df_botptE['bottom_pressure'].astype(float) * 0.670
#MJ03F_cal_depths = [MJ03F_pressure * 0.0670 for MJ03F_pressure in MJ03F_pressure]
#list comprehention
epoch= [i.timestamp() for i in df_botptE.index.to_pydatetime()]
df_botptE['epoch'] = epoch
df_botptE= df_botptE.sort_index()

In [ ]:
df_botpt.head()

In [ ]:
df_botptE.head()

In [ ]:
df_botpt.tail()

In [ ]:
df_botptE.tail()

In [ ]:
del df_botptE['epoch']
del df_botptE['bottom_pressure']
del df_botpt['epoch']
del df_botpt['bottom_pressure']

In [ ]:
x = df_botptE.head()
y = df_botpt.head()

In [ ]:
z = pd.merge(x,y,how='outer', indicator=True, left_index=True, right_index=True)

In [ ]:
test = pd.merge(df_botpt, df_botptE,how='outer', indicator=True, left_index=True, right_index=True, suffixes=('_F', '_E'))

In [ ]:
df_botptMerge = test[test['_merge'] == 'both']
del df_botptMerge['_merge']

In [ ]:
df_botptMerge = df_botptMerge.loc['2017-01-1 00:00:00':'2018-10-31 00:00:00']

In [ ]:
# df_botpt[df_botpt['_merge'] == 'right_only']

# df_botpt[df_botpt['_merge'] == 'left_only']

#### Calculate Depth difference 

In [ ]:
depthDiff = df_botptMerge['depth_E'].values - df_botptMerge['depth_F'].values

https://stackoverflow.com/questions/23199796/detect-and-exclude-outliers-in-pandas-dataframe

https://stackoverflow.com/questions/35829364/fixing-inflexion-point-estimate-using-python

In [ ]:
df_botptMerge['diff'] = depthDiff

In [ ]:
h2 = df_botptMerge['depth_F'].tolist()
h1 = df_botptMerge['depth_E'].tolist()


In [ ]:
def subtract_lists(h1, h2):
    diff = [a - b for a,b in zip(h1, h2)]
    return diff

In [ ]:
height = subtract_lists(h1,h2)

In [ ]:
# time = list(df_botptMerge.index.values)
#height = x.tolist()
#height = df_botptMerge['diff'].tolist()
time_int = []
time = list(pd.to_datetime(df_botptMerge.index.values))
for i in time:
    i = np.datetime64(i).astype(datetime.datetime)
    time_int.append(dates.date2num(i))

In [ ]:
plt.close()
fig, ax = plt.subplots()
fig.set_size_inches(28, 6)
hb1 = ax.plot(time_int, height)
ax.yaxis.grid(True)
ax.xaxis.grid(True)
ax1.set_xlim(datetime.datetime(2017, 1, 1, 0, 0),datetime.datetime(2018, 10, 31, 0, 0))
years = dates.YearLocator()
months = dates.MonthLocator()
yearsFmt = dates.DateFormatter('\n\n\n%Y')
monthsFmt = dates.DateFormatter('%b')
ax.xaxis.set_major_locator(months)
ax.xaxis.set_major_formatter(monthsFmt)
ax.xaxis.set_minor_locator(years)
ax.xaxis.set_minor_formatter(yearsFmt)
plt.tight_layout()
plt.setp(ax.xaxis.get_majorticklabels(), rotation=90)
#plt.gca().invert_yaxis()
plt.show()

In [ ]:
weird_data = np.array(height)
time_array = np.array(time)

cleaner_data = []
cleaner_time = []

# i = 0
# while i < len(weird_data):
#     if (weird_data[i] - weird_data[i-1] > 2

length = len(weird_data)

i = 0
while i < length:
    j = i+2
    while j < length:
        if np.abs(weird_data[i] - weird_data[j]) < .1:
            cleaner_data.append(weird_data[j])
            cleaner_time.append(time_array[j])
            break
        else:
            j += 1
    i = j


In [ ]:
plt.close()
fig3, (ax1,ax2) = plt.subplots(2,1)
fig3.set_size_inches(28, 6)
hb1 = ax1.plot(time_int, height)
ax1.yaxis.grid(True)
ax1.xaxis.grid(True)
ax1.set_xlim(datetime.datetime(2017, 1, 1, 0, 0),datetime.datetime(2018, 10, 31, 0, 0))
years = dates.YearLocator()
months = dates.MonthLocator()
yearsFmt = dates.DateFormatter('\n\n\n%Y')
monthsFmt = dates.DateFormatter('%b')
ax1.xaxis.set_major_locator(months)
ax1.xaxis.set_major_formatter(monthsFmt)
ax1.xaxis.set_minor_locator(years)
ax1.xaxis.set_minor_formatter(yearsFmt)

hb1 = ax2.plot(cleaner_time, cleaner_data)
ax2.yaxis.grid(True)
ax2.xaxis.grid(True)
ax2.set_xlim(datetime.datetime(2017, 1, 1, 0, 0),datetime.datetime(2018, 10, 31, 0, 0))
years = dates.YearLocator()
months = dates.MonthLocator()
yearsFmt = dates.DateFormatter('\n\n\n%Y')
monthsFmt = dates.DateFormatter('%b')
ax2.xaxis.set_major_locator(months)
ax2.xaxis.set_major_formatter(monthsFmt)
ax2.xaxis.set_minor_locator(years)
ax2.xaxis.set_minor_formatter(yearsFmt)

plt.tight_layout()
plt.setp(ax1.xaxis.get_majorticklabels(), rotation=90)
#plt.gca().invert_yaxis()
plt.show()

##### This method appears to do a pretty good of eliminating the tidal signal. You can clearly see the periods of deflation near 9/2016, 2/2017, 1/2018, and 7/2018

##### There is more work to do to eliminate the dropouts and calculate the differing rates of inflation or deflation. 

In [ ]:
plt.savefig('Figures/1dayDiff_2017_2018.png')

In [ ]:
df_botptMerge.head()

In [ ]:
x=df_botptMerge[np.abs(df_botptMerge['diff']-df_botptMerge['diff'].mean()) <= (df_botptMerge['diff'].std())]
# from scipy import stats
# df_botptMerge[(np.abs(stats.zscore(df_botptMerge)) < 3).all(axis=1)]

In [ ]:
x.head()

In [ ]:
height = x['diff'].tolist()
time_int = []
time = list(pd.to_datetime(x.index.values))
for i in time:
    i = np.datetime64(i).astype(datetime.datetime)
    time_int.append(dates.date2num(i))

In [ ]:
plt.close()
fig, ax = plt.subplots()
fig.set_size_inches(28, 6)
hb1 = ax.plot(time_int, height)
ax.yaxis.grid(True)
ax.xaxis.grid(True)
ax.set_xlim(datetime.datetime(2017, 1, 1, 0, 0),datetime.datetime(2018, 10, 31, 0, 0))
years = dates.YearLocator()
months = dates.MonthLocator()
yearsFmt = dates.DateFormatter('\n\n\n%Y')
monthsFmt = dates.DateFormatter('%b')
ax.xaxis.set_major_locator(months)
ax.xaxis.set_major_formatter(monthsFmt)
ax.xaxis.set_minor_locator(years)
ax.xaxis.set_minor_formatter(yearsFmt)
plt.tight_layout()
plt.setp(ax.xaxis.get_majorticklabels(), rotation=90)
#plt.gca().invert_yaxis()
plt.show()

In [ ]:
plt.close()
fig, ax = plt.subplots()
fig.set_size_inches(28, 6)
hb1 = ax.plot(time_int, height, vmin=0, vmax=30, gridsize=(1500,100), mincnt=1, cmap='Greens', linewidths=0)
fig.colorbar(hb1, pad = 0.01)
ax.yaxis.grid(True)
ax.xaxis.grid(True)
ax.set_xlim(datetime.datetime(2017, 11, 1, 0, 0),datetime.datetime(2018, 8, 31, 0, 0))
years = dates.YearLocator()
months = dates.MonthLocator()
yearsFmt = dates.DateFormatter('\n\n\n%Y')
monthsFmt = dates.DateFormatter('%b')
ax.xaxis.set_major_locator(months)
ax.xaxis.set_major_formatter(monthsFmt)
ax.xaxis.set_minor_locator(years)
ax.xaxis.set_minor_formatter(yearsFmt)
plt.tight_layout()
plt.setp(ax.xaxis.get_majorticklabels(), rotation=90)
#plt.gca().invert_yaxis()
plt.show()

#### Zoom in on 2018

In [ ]:
plt.savefig('Figures/2018.png')

In [ ]:
df_depthDiff = pd.DataFrame(depthDiff)


In [ ]:
df_botptMerge.head()

In [ ]:
from scipy.interpolate import UnivariateSpline

#raw data
data = depthDiff

plt.plot(np.gradient(data), '+')

spl = UnivariateSpline(np.arange(len(data)), np.gradient(data), k=5)
spl.set_smoothing_factor(1000)
plt.plot(spl(np.arange(len(data))), label='Smooth Fct 1e3')
spl.set_smoothing_factor(10000)
plt.plot(spl(np.arange(len(data))), label='Smooth Fct 1e4')
plt.legend(loc='lower left')
#plt.set_xlim(datetime.datetime(2017, 11, 1, 0, 0),datetime.datetime(2018, 8, 31, 0, 0))
max_idx = np.argmax(spl(np.arange(len(data))))
plt.vlines(max_idx, -.25, .25, linewidth=5, alpha=0.3)

In [ ]:
import scipy.optimize as so
F = lambda x: -spl(x)
so.fmin(F, 102)
Optimization terminated successfully.
         Current function value: -3.339112
         Iterations: 20
         Function evaluations: 40